In [48]:
import pandas as pd
import pathlib
import os
import re

# 1) Get all participants from respective datasets

In [72]:
# for both sheets in excel
df = pd.read_excel("data/english-results-data.xlsx", sheet_name="Time 1")
df1 = pd.read_excel("data/english-results-data.xlsx", sheet_name="Repeats")

FileNotFoundError: [Errno 2] No such file or directory: 'data/english-results-data.xlsx'

In [ ]:
print(df["WAB Type"].unique())
print(df1["WAB Type"].unique())

In [ ]:
def get_participants_files(wab_type, data1, data2):
    data1 = data1.loc[data1["WAB Type"] == wab_type]
    participants = set(data1["Participant ID"])
    data2 = data2.loc[data2["WAB Type"] == wab_type]
    participants1 = set(data2["Participant ID"])
    
    participants.update(participants1)
    ids = list(participants)
    ids.sort(key=str.lower)
    return ids

In [ ]:
# number of brocas patients
ids = get_participants_files("Broca", df, df1)
ids

In [ ]:
w_ids = get_participants_files("Wernicke", df, df1)
t_ids = get_participants_files("TransSensory", df, df1)
c_ids = get_participants_files("Conduction", df, df1)
a_ids = get_participants_files("Anomic", df, df1)
a1_ids = get_participants_files("Anomic ", df, df1)

In [ ]:
w_ids

# 2) Get file names we are interested in for PWA

In [ ]:
def get_file_names(wab_type, data1, data2):
    ids = get_participants_files(wab_type, data1, data2)
    brocas = [] # to check for missing data
    filenames = [] # to keep track of file names
    
    lower_ids = [x.lower() for x in ids]
    
    data_dir = str(pathlib.Path().resolve()) + "\\data\\data_pwa\\"
    # print(data_dir)
    
    for subdir, dirs, files in os.walk(data_dir):
        for file in files:
            i = file.replace(".cha","")
            dirname = os.path.basename(subdir).lower()
 
            # each naming convention is different
            if (dirname == "fridriksson-2" or dirname == "umd") and dirname + '-' + i.lower() in lower_ids:
                brocas.append(dirname + '-' + i)
                filenames.append(file)
            
            if i.lower() in lower_ids:
                brocas.append(i)
                filenames.append(file)
            if dirname+i.lower() in lower_ids:
                brocas.append(dirname+i)
                filenames.append(file)
    print(f"Found datafiles: {len(brocas)}")
    print(f"Actual datafiles in excel: {len(ids)}")
    
    lower_brocas= [x.lower() for x in brocas]
    
    print(f"Missing datafiles: {set(lower_ids) - set(lower_brocas)}")
    # missing data
    print(f"Number of missing datafiles: {len(set(lower_ids) - set(lower_brocas))}")
    
    return filenames

In [ ]:
WAB = ["Wernicke", "TransSensory", "Conduction", "Anomic", "Anomic "] 
fname = ["wernicke_fname.csv", "transsensory_fname.csv", "conduction_fname.csv", "anomic_fname.csv", "anomic1_fname.csv"]

for type, file in zip(WAB, fname):
    print("WAB: ", type)
    filenames = get_file_names(type, df, df1)
    print("------------------")
    aphasia = pd.DataFrame(filenames).T
    aphasia.to_csv("data/"+file, index=False, header=False)

In [ ]:
WAB = "Broca" 
fname = "broca_fname.csv"

filenames = get_file_names(WAB,df, df1)

file_broca = pd.DataFrame(filenames).T
file_broca.to_csv(fname, index=False, header=False)

# 3) Getting the data preprocessed
#### Run setup_dataframe.py and then preprocessing_edited.py

# 4) Section the preprocessed data into scenarios (merge sentences) and combine broca data and control data

In [51]:
broca_fname = "../datafiles/clan_data/preprocessed_broca.csv"
control_fname = "../datafiles/clan_data/preprocessed_control.csv"

broca_data = pd.read_csv(broca_fname).dropna()
control_data = pd.read_csv(control_fname, encoding='utf8').dropna()

In [52]:
broca_patients = len(broca_data["source_file"].unique())
control = len(control_data["source_file"].unique())
print(f"Brocas patients: {broca_patients}, control: {control}")

Brocas patients: 173, control: 346


In [53]:
def merge_scenario(data, label):
    columns = ['source_file', 'scenario', 'preprocessed_text', 'label']
    new_data = [] # pd.DataFrame(columns=columns)
    text = ""
    n = -1
    
    for index, row in data.iterrows():
        # print(f"index : {index}, scenario {row['scenario']}")
        if index == 0:
            prev_scenario = row["scenario"] 
            prev_sourcefile = row["source_file"]
        current_scenario = row["scenario"]
        current_sourcefile = row["source_file"]
        
        if n!= 2 and current_scenario == prev_scenario and current_sourcefile == prev_sourcefile:
            # print("same scenario")
            # print(text)
            # print()
            text += " " + row["preprocessed_text"]
            n += 1
            
        else:
            n = 0
            # print()
            text = str(re.sub(' +', ' ', text)).lstrip().rstrip().replace('?','.').replace('!', '.')
            if len(re.findall('\.',text)) == 3:
                new_data.append([row["source_file"], prev_scenario, text, label])
            text = row["preprocessed_text"]
        
        prev_scenario = current_scenario
        prev_sourcefile = current_sourcefile
        
        # if index == 74:
        #     break
    
    return pd.DataFrame(new_data, columns=columns)

In [54]:
# LABEL 1 FOR BROCA 0 FOR NOT APHASIC, for classifier
broca_data_merge = merge_scenario(broca_data, 1)
control_data_merge = merge_scenario(control_data, 0)
data_full_scenario = pd.concat([broca_data_merge, control_data_merge], ignore_index=True)
data_full_scenario["preprocessed_text"] = [x.lstrip().rstrip().lower() for x in data_full_scenario['preprocessed_text']]

In [56]:
# A bit imbalanced?
print(len(broca_data_merge))
print(len(control_data_merge))

4888
12992


In [57]:
# shuffle data
data_full_scenario = data_full_scenario.sample(frac=1).reset_index(drop=True)

In [58]:
data_full_scenario

,source_file,scenario,preprocessed_text,label
0,wright61a.cha,Window,and it threw it flew the ball flew through the...,0
1,wright29a.cha,Window,he looked out the window. was and was looking ...,0
2,wright59a.cha,Cinderella,but the stepmother also had two girls. so they...,0
3,richardson35.cha,Cat,"alright. so the story starts out, the little g...",0
4,capilouto65a.cha,Important_Event,and when they got when a certain higher grade ...,0
...,...,...,...,...
17875,capilouto48a.cha,Cinderella,so in some way she ends up crying. and she is ...,0
17876,wright93a.cha,Cinderella,and it all get they help her put her dress on ...,0
17877,UMD06.cha,Cat,she has a bicycle. and it is probably the dadd...,0
17878,capilouto47a.cha,Important_Event,my mother hadta go to work. we got this ugly g...,0


In [59]:
data_full_scenario.to_csv("../datafiles/clan_data/preprocessed_merge_sentence.csv")

In [61]:
data_full_scenario[data_full_scenario["label"]==1].to_csv("../datafiles/clan_data/sentence_broca.csv")

# 5) Alternative: don't merge scenarios

In [ ]:
import pandas as pd
broca_fname = "data/preprocessed_broca.csv"
control_fname = "data/preprocessed_control.csv"

broca_data = pd.read_csv(broca_fname).dropna()
control_data = pd.read_csv(control_fname, encoding='utf8').dropna()

print(broca_data.columns)
broca_data["label"] = 1
control_data["label"] = 0

data_full_scenario = pd.concat([broca_data, control_data], ignore_index=True)
data_full_scenario = data_full_scenario.sample(frac=1).reset_index(drop=True)
data_full_scenario["preprocessed_text"] = [x.lstrip().rstrip().lower() for x in data_full_scenario['preprocessed_text']]

data_full_scenario.to_csv("data/processed_merge.csv")

# Spoken corpora for simulation

In [27]:
import pandas as pd
import re
df = pd.read_csv("../datafiles/spoken corpus/boston/preprocessed_test.csv").drop(["Unnamed: 0"], axis=1).dropna().reset_index()
df = df[df["preprocessed_text"]!="."]

In [28]:
df

,index,text,speaker,source_file,original_text,preprocessed_text
0,1,&{n=FOOTSTEPS Okay . 3400_4100,*REBE,SBC008.cha,&{n=FOOTSTEPS Okay . 3400_4100,okay.
2,4,The +/. 7400_7800,*REBE,SBC008.cha,The +/. 7400_7800,the.
3,5,(..) The way that your testimony is &}n=FOOTST...,*REBE,SBC008.cha,(..) The way that your testimony is &}n=FOOTST...,the way that your testimony is coming in
4,6,(..) I don't know if I explained this to you ...,*REBE,SBC008.cha,(..) I don't know if I explained this to you ...,i do not know if i explained this to you on th...
6,8,is 12010_12410,*REBE,SBC008.cha,is 12010_12410,is
...,...,...,...,...,...,...
57862,61142,(..) oh further than that 1483042_1484999,*JON,SBC060.cha,(..) oh further than that 1483042_1484999,oh further than that
57863,61143,maybe nineteen-seventy . 1484999_1486347,*JON,SBC060.cha,maybe nineteen-seventy . 1484999_1486347,maybe nineteen seventy.
57864,61144,(..) &=in (.) in ʔuh +/. 1486347_1488519,*JON,SBC060.cha,(..) &=in (.) in ʔuh +/. 1486347_1488519,in.
57865,61145,in Houston 1488519_1489281,*JON,SBC060.cha,in Houston 1488519_1489281,in houston


In [29]:
df.iloc[8]

index                                                              13
text                  where he: acted in the same way ? 18860_20560
speaker                                                         *REBE
source_file                                                SBC008.cha
original_text         where he: acted in the same way ? 18860_20560
preprocessed_text                     where he acted in the same way?
Name: 10, dtype: object

In [30]:
def merge_sentence():
    columns = ['source_file', 'speaker', 'preprocessed_text', 'label']
    new_data = [] # pd.DataFrame(columns=columns)
    sentence = ""
    label = 0
    for index, row in df.iterrows():
        
        # if index == 25:
        #     return pd.DataFrame(new_data, columns=columns)
        
        curr_sent = row["preprocessed_text"]
        # print(curr_sent)
        
        if index == 0:
            prev_speaker = row["speaker"] 
            prev_sourcefile = row["source_file"]
        current_speaker = row["speaker"]
        current_sourcefile = row["source_file"]

        # if continued sentence in same speeaker then add 
        if prev_speaker == current_speaker and prev_sourcefile == current_sourcefile:
            sentence += " " + curr_sent

        # if speaker changes or file changes between sentence 
        elif (current_speaker != prev_speaker or current_sourcefile != prev_sourcefile) and sentence != "":
            sentence += "."

        # if conversation changes at new sentence
        elif (current_speaker != prev_speaker or current_sourcefile != prev_sourcefile) and sentence == "":
            sentence += " " + curr_sent
        
        # if sentence over then create new sent
        if any(x in sentence for x in [".", "!", "?"]):
            new_data.append([row["source_file"], prev_speaker, str(re.sub(' +', ' ', sentence)), label])
            # print(sentence)
            # print()
            sentence = ""
            
        prev_speaker = current_speaker
        prev_sourcefile = current_sourcefile
        
    return pd.DataFrame(new_data, columns=columns)

In [31]:
df1 = merge_sentence()

In [32]:
df1

,source_file,speaker,preprocessed_text,label
0,SBC008.cha,*REBE,okay.,0
1,SBC008.cha,*REBE,the.,0
2,SBC008.cha,*REBE,the way that your testimony is coming in i do...,0
3,SBC008.cha,*REBE,is when a person is charged with multiple cri...,0
4,SBC008.cha,*REBE,when we have to prove in this case we have to...,0
...,...,...,...,...
33398,SBC060.cha,*JON,but.,0
33399,SBC060.cha,*ALAN,these.,0
33400,SBC060.cha,*JON,these.,0
33401,SBC060.cha,*JON,what astounds me is and this goes back into n...,0


In [39]:
import re
sentences = df1["preprocessed_text"]
x = pd.Series([len(re.findall("[a-zA-Z_]+", x)) for x in sentences])
df1.insert(loc=0, column="lens", value=x)
df1 = df1[df1['lens'] >= 3]

ValueError: cannot insert lens, already exists

In [40]:
df1 = df1.reset_index()

## Merge at para level

In [35]:
import pandas as pd

In [41]:
def merge_scenario(data, label):
    columns = ['source_file', 'speaker', 'preprocessed_text', 'label']
    new_data = [] # pd.DataFrame(columns=columns)
    text = ""
    num = -1
    for index, row in data.iterrows():
        print(f"index : {index}, speaker {row['speaker']}")
        if index == 0:
            prev_scenario = row["speaker"] 
            prev_sourcefile = row["source_file"]
        current_scenario = row["speaker"]
        current_sourcefile = row["source_file"]
        
        # if index == 20:
        #     break
        # if current_scenario == prev_scenario and current_sourcefile == prev_sourcefile:
        if num != 2 and current_sourcefile == prev_sourcefile:
            # print("same scenario")
            # print(text)
            # print()
            text += " " + row["preprocessed_text"].lstrip().rstrip().replace('?','.').replace('!', '.')
            num += 1

        else:
            # print("new scenario")
            # print(text)
            # print()
            num = 0
            text = str(re.sub(' +', ' ', text)).lstrip().rstrip().replace('?','.').replace('!', '.')
            if len(re.findall('\.',text)) == 3:
                new_data.append([row["source_file"], prev_scenario, text, label])
            text = row["preprocessed_text"]
        
        prev_scenario = current_scenario
        prev_sourcefile = current_sourcefile
        
        # if index == 74:
        #     break
    
    return pd.DataFrame(new_data, columns=columns)

In [42]:
df1 = df1[df1['preprocessed_text'].notna()]
df1

,index,lens,source_file,speaker,preprocessed_text,label
0,2,21,SBC008.cha,*REBE,the way that your testimony is coming in i do...,0
1,3,21,SBC008.cha,*REBE,is when a person is charged with multiple cri...,0
2,4,14,SBC008.cha,*REBE,when we have to prove in this case we have to...,0
3,5,9,SBC008.cha,*ENV,to expose himself to a person for sexual arou...,0
4,6,15,SBC008.cha,*REBE,when we have to prove something like specific...,0
...,...,...,...,...,...,...
23202,33394,6,SBC060.cha,*ALAN,you had to look close xx.,0
23203,33395,3,SBC060.cha,*ALAN,pretty good indication.,0
23204,33396,3,SBC060.cha,*JON,pretty good indication.,0
23205,33397,3,SBC060.cha,*ALAN,well you know.,0


In [43]:
new_data = merge_scenario(df1, 0)

index : 0, speaker *REBE
index : 1, speaker *REBE
index : 2, speaker *REBE
index : 3, speaker *ENV
index : 4, speaker *REBE
index : 5, speaker *REBE
index : 6, speaker *REBE
index : 7, speaker *REBE
index : 8, speaker *REBE
index : 9, speaker *REBE
index : 10, speaker *REBE
index : 11, speaker *REBE
index : 12, speaker *RICK
index : 13, speaker *RICK
index : 14, speaker *REBE
index : 15, speaker *RICK
index : 16, speaker *RICK
index : 17, speaker *RICK
index : 18, speaker *REBE
index : 19, speaker *RICK
index : 20, speaker *REBE
index : 21, speaker *REBE
index : 22, speaker *REBE
index : 23, speaker *RICK
index : 24, speaker *REBE
index : 25, speaker *RICK
index : 26, speaker *RICK
index : 27, speaker *RICK
index : 28, speaker *REBE
index : 29, speaker *RICK
index : 30, speaker *REBE
index : 31, speaker *RICK
index : 32, speaker *RICK
index : 33, speaker *REBE
index : 34, speaker *RICK
index : 35, speaker *RICK
index : 36, speaker *RICK
index : 37, speaker *REBE
index : 38, speaker *RE

In [47]:
new_data

,source_file,speaker,preprocessed_text,label
0,SBC008.cha,*REBE,the way that your testimony is coming in i do ...,0
1,SBC008.cha,*REBE,to expose himself to a person for sexual arous...,0
2,SBC008.cha,*REBE,in at least one of the cases. then we are allo...,0
3,SBC008.cha,*REBE,things like that that is why we are able to ha...,0
4,SBC008.cha,*REBE,well i have made two one a year from almost a ...,0
...,...,...,...,...
7713,SBC060.cha,*ALAN,on areas score score higher on the test than t...,0
7714,SBC060.cha,*ALAN,i am told i have not seen any of them but the ...,0
7715,SBC060.cha,*JON,but they are well educated people down there. ...,0
7716,SBC060.cha,*ALAN,i am s. i know that this fellow goldstone who ...,0


In [45]:
new_data.to_csv("../datafiles/spoken corpus/boston/preprocessed_boston.csv")